In [1]:
import numpy as np
import pandas as pd
import time

## READ THE DATA

In [2]:
def SplitFile(fileLines):
    g = []
    for line in fileLines:
        if line == '':
            yield g
            g = []
        else:
            g.append(line)
    yield g

In [3]:
fileName = 'input.txt'
fileLines = open(fileName).read().splitlines()
rules, messages = list(SplitFile(fileLines))

## PART 1

In [4]:
indexToRuleDictionary = {}

for rule in rules:
    ruleIndex, ruleCondition = rule.split(": ")
    indexToRuleDictionary[int(ruleIndex)] = ruleCondition

In [5]:
def AppendToRuleDictionary(dictionary, key, element):
    if key in dictionary:
        dictionary[key].append(element)
    else:
        dictionary[key] = element

In [6]:
## This one decodes any rule
def DecodeRule(indexToRuleDictionary, processedRuleDictionary, ruleString):
    ## Get index of pipe, or -1 if there is none
    splitIndex = next((index for index, element in enumerate(ruleString) if element == '|'), -1)
    
    ## Case 1: rule is a single character ("a", or "b")
    if ((splitIndex == -1) and (ruleString[0] == '"')):
        return [ruleString[1:-1]]
    
    ## Case 2: rule is an index or a series of indexes with no pipes ('4 1 5', '3')
    if ((splitIndex == -1) and (ruleString[0] != '"')):
        ruleList = [int(element) for element in ruleString.split(' ')]
        decodedRuleList = ['']
        for ruleIndex in ruleList:
            newDecodedRuleList = DecodeRuleByIndex(indexToRuleDictionary, processedRuleDictionary, ruleIndex)
            decodedRuleList = [str(before) + str(after) for before in decodedRuleList for after in newDecodedRuleList]
        return decodedRuleList
    
    ## Case 3: rule is two series of indexes split by a pipe ('4 5 | 5 4')
    leftRule = ruleString[:splitIndex-1]
    rightRule = ruleString[splitIndex+2:]
    lstLeft = DecodeRule(indexToRuleDictionary, processedRuleDictionary, leftRule)
    lstRight = DecodeRule(indexToRuleDictionary, processedRuleDictionary, rightRule)
    return lstLeft + lstRight

## This one decodes any indexed rule, updating the processed rule dictionary
def DecodeRuleByIndex(indexToRuleDictionary, processedRuleDictionary, indexToDecode):
    if indexToDecode in processedRuleDictionary:
        return processedRuleDictionary[indexToDecode]
    
    ruleString = indexToRuleDictionary[indexToDecode]
    decodedRule = DecodeRule(indexToRuleDictionary, processedRuleDictionary, ruleString)
    AppendToRuleDictionary(processedRuleDictionary, indexToDecode, decodedRule)
    return decodedRule

In [7]:
dicProcess = {}
tic = time.perf_counter()
DecodeRuleByIndex(indexToRuleDictionary, dicProcess, 0)
matchCounter = sum([True for message in messages if message in dicProcess[0]])
toc = time.perf_counter()
        
print('Number of messages that match rule 0:', matchCounter)
print(f"Total found in {toc - tic:0.4f} seconds.")

Number of messages that match rule 0: 104
Total found in 10.9112 seconds.


## PART 2
Clearly the Part 2 instructions heavily punished my tactic of generating all the matchable messages: even if I limited the generation to the maximum length of the input messages, generating all possible combinations matching rule 8 alone would take ages.

Therefore I opted for an alternative approach: essentially, every rule boils down to a sequence of two possible rules (the one for "a" and the one for "b"). I should simply be able follow the recursion down to its fundamental blocks... except there are those damn looping rules.

*But!!* We don't actually have to follow the recursion all the way down. Suppose we have something like "4, 5, 1". In the test input for Part 2, rule 4 requires a message beginning with a sub-message matching rule 1, which in turn is just the "a" rule. If the message I'm checking begins with a *b*, all the recursion in the world will never be able to produce the message since the first character is wrong. If it *isn't* wrong, I can strip it from the message, remove the first rule 1 instance and retry. Either I come to an empty message with no rules left to apply (the message could be generated with the starting rules) or I don't (the message could not be generated). Pipes just mean more forks, no big deal.

In [8]:
indexToRuleDictionary[8] = '42 | 42 8'
indexToRuleDictionary[11] = '42 31 | 42 11 31'

In [9]:
def RuleStringToList(ruleString):
    splitIndex = next((index for index, element in enumerate(ruleString) if element == '|'), -1)
    
    ## Case 1: rule is a single character ("a", or "b")
    if ruleString[0] == '"':
        return ruleString
    ## Case 2: rule is an index or a series of indexes with no pipes ('4 1 5', '3')
    elif splitIndex == -1:
        return [[int(element) for element in ruleString.split(' ')]]
    ## Case 3: rule is two series of indexes split by a pipe ('4 5 | 5 4')
    else:
        leftRule = RuleStringToList(ruleString[:splitIndex-1])
        rightRule = RuleStringToList(ruleString[splitIndex+2:])
        ## I know it's dumb, but it works
        return [leftRule[0], rightRule[0]]

def RulesCanProduceMessage(message, rulesList):   
    if message == '' and rulesList == []:
        return True
    elif message == '' or rulesList == []:
        return False
    
    firstRule = rulesList[0]
    firstRuleRequirements = RuleStringToList(indexToRuleDictionary[firstRule])
    
    ## If the first rule is one of the two "single letter" rules, i.e. #: "a" or #: "b"
    if '"' in firstRuleRequirements:
        ## If the message begins with the matching letter, strip character and corresponding rule
        if message[0] in firstRuleRequirements:
            return RulesCanProduceMessage(message[1:], rulesList[1:])
        ## If it doesn't, it will never match
        else:
            return False
    ## If not a single letter rule, then we expand the first rule to its requirements and rerun the function
    ## RuleStringToList is coded as to return [[1, 2]] if rule is "#: 1 2" and [[1, 2], [3,4]] if rule is "#: 1 2 | 3 4"
    else:
        matches = False
        for nestedRule in firstRuleRequirements:
            matches += RulesCanProduceMessage(message, nestedRule + rulesList[1:])
        return matches

In [10]:
tic = time.perf_counter()
matchCounterPatched = sum(RulesCanProduceMessage(m,[0]) for m in messages)
toc = time.perf_counter()

print('Number of messages that match rule 0 with patched rules 8 and 11:', matchCounterPatched)
print(f"Total found in {toc - tic:0.4f} seconds.")

Number of messages that match rule 0 with patched rules 8 and 11: 314
Total found in 3.8873 seconds.
